In [1]:
# initial imports
import pandas as pd
from pathlib import Path

In [2]:
#set file path
file_path = Path("C:/Users/gerar/OneDrive/Desktop/home_test/limit_data.csv")
# create dataframe
df = pd.read_csv(file_path)

In [3]:
# checking file
df.head(5)

,Strategy,Product,Internal Limit in coin,Product.1,Currency,Price,Instrument,Coin,Exchange position limit notional ( in BTC),Portfolio Limit (in USD),Limit Per Individual Product (in USD)
0,S4,ZIL,366020.000,ICP,USD,4.950000,ZIL_exchange,ZIL,1.56,"1,600,000","75,000"
1,S4,BTC,2.835,SRM,USD,0.717000,BTC_exchange,BTC,0.52,NaN,NaN
2,S4,USDT,139315.340,ICX,USD,0.214200,USDT_exchange,USDT,0.21,NaN,NaN
3,S4,BUSD,107574.305,XLM,USD,0.112900,BUSD_exchange,BUSD,0.52,NaN,NaN
4,S4,XRP,87968.500,BTT,USD,0.002777,XRP_exchange,XRP,1.04,NaN,NaN


In [4]:
# create new df for product and internal limit in coin (the first tables in the excel sheet)

internal_df = df[['Product','Internal Limit in coin']].copy()

In [5]:
#Check ( we didnt include the strategy column since its not needed)
internal_df.head()

,Product,Internal Limit in coin
0,ZIL,366020.000
1,BTC,2.835
2,USDT,139315.340
3,BUSD,107574.305
4,XRP,87968.500


In [6]:
# create new df for prices ( second table in excel sheet)
price_df = df[['Product.1','Price']].copy()

In [7]:
#check
price_df.head()

,Product.1,Price
0,ICP,4.950000
1,SRM,0.717000
2,ICX,0.214200
3,XLM,0.112900
4,BTT,0.002777


In [8]:
# Rename Column in order to create merge (we need a column named identically in order to do merge /vlookup)
price_df = price_df.rename(columns={'Product.1': 'Product'})

In [9]:
#check
price_df

,Product,Price
0,ICP,4.950000
1,SRM,0.717000
2,ICX,0.214200
3,XLM,0.112900
4,BTT,0.002777
...,...,...
357,HIGH,1.401000
358,MOVR,9.930000
359,GLMR,0.430800
360,BETH,1272.517239


In [10]:
#create inner join merge , equivalent of VLOOKUP
inner_join = pd.merge(internal_df, price_df, on ='Product', how ='inner') 

In [11]:
#check
inner_join

,Product,Internal Limit in coin,Price
0,ZIL,366020.000,0.03000
1,BTC,2.835,19176.69000
2,BTC,1.200,19176.69000
3,BTC,1.655,19176.69000
4,BTC,0.895,19176.69000
...,...,...,...
71,COTI,73126.500,0.09680
72,CLV,109648.500,0.09795
73,CHR,63457.500,0.13950
74,API3,2874.000,1.52500


In [13]:
#create new column by obtaining the product (*) of "price" and "internal limit in coin"
inner_join['USD Value'] = inner_join['Internal Limit in coin'] * inner_join ['Price']

In [14]:
#check - now we had a USD Value for all internal coins.
inner_join.head(30)

,Product,Internal Limit in coin,Price,USD Value
0,ZIL,366020.000,0.03000,10980.600000
1,BTC,2.835,19176.69000,54365.916150
2,BTC,1.200,19176.69000,23012.028000
3,BTC,1.655,19176.69000,31737.421950
4,BTC,0.895,19176.69000,17163.137550
5,BTC,1.235,19176.69000,23683.212150
6,USDT,139315.340,1.00000,139315.340000
7,USDT,91589.225,1.00000,91589.225000
8,USDT,70516.560,1.00000,70516.560000
9,USDT,81770.505,1.00000,81770.505000


In [15]:
#create pivot table aggregating by product - we are adding all the values.
pivot_table = pd.pivot_table(inner_join, values='USD Value', index='Product', aggfunc='sum')

pivot_table.head(20)

,USD Value
Product,
ADA,28333.649450
ALGO,13869.384000
API3,4382.850000
AUDIO,10665.625500
AVAX,22366.540000
BAKE,13431.617100
BLZ,6536.388300
BOND,11479.840000
BSW,6558.675000


In [16]:
#this is the total internal value in USD
pivot_table['USD Value'].sum()

1770395.4349250547

In [20]:
# Are we within our internal overall portfolio limit ? - Yes we are over by roughly 170 k.
(1770395.43 - 1600000)

170395.42999999993

In [21]:
exchange = df[['Instrument','Coin','Exchange position limit notional ( in BTC)']].copy()

In [22]:
#check
exchange.head(20)

,Instrument,Coin,Exchange position limit notional ( in BTC)
0,ZIL_exchange,ZIL,1.56
1,BTC_exchange,BTC,0.52
2,USDT_exchange,USDT,0.21
3,BUSD_exchange,BUSD,0.52
4,XRP_exchange,XRP,1.04
5,THETA_exchange,THETA,0.52
6,RNDR_exchange,RNDR,0.52
7,REEF_exchange,REEF,0.52
8,QUICK_exchange,QUICK,0.52
9,LINK_exchange,LINK,7.82


In [23]:
# create new column to display USD Value by multiplying BTC price given.
exchange['USD Value'] = df['Exchange position limit notional ( in BTC)'] * 19176.69

In [24]:
#check
exchange

,Instrument,Coin,Exchange position limit notional ( in BTC),USD Value
0,ZIL_exchange,ZIL,1.56,29915.6364
1,BTC_exchange,BTC,0.52,9971.8788
2,USDT_exchange,USDT,0.21,4027.1049
3,BUSD_exchange,BUSD,0.52,9971.8788
4,XRP_exchange,XRP,1.04,19943.7576
...,...,...,...,...
357,NaN,NaN,NaN,NaN
358,NaN,NaN,NaN,NaN
359,NaN,NaN,NaN,NaN
360,NaN,NaN,NaN,NaN


In [25]:
#drop nulls
exchange.dropna(inplace=True)

In [26]:
#check
exchange


,Instrument,Coin,Exchange position limit notional ( in BTC),USD Value
0,ZIL_exchange,ZIL,1.56,29915.6364
1,BTC_exchange,BTC,0.52,9971.8788
2,USDT_exchange,USDT,0.21,4027.1049
3,BUSD_exchange,BUSD,0.52,9971.8788
4,XRP_exchange,XRP,1.04,19943.7576
...,...,...,...,...
59,COTI_exchange,COTI,1.56,29915.6364
60,CLV_exchange,CLV,0.52,9971.8788
61,CHR_exchange,CHR,0.52,9971.8788
62,API3_exchange,API3,0.52,9971.8788


In [30]:
#Sorting by Coin
exchange.sort_values(by=['Coin'],inplace=True)

In [31]:
exchange

,Instrument,Coin,Exchange position limit notional ( in BTC),USD Value
21,ADA_exchange,ADA,0.52,9971.8788
20,ALGO_exchange,ALGO,1.56,29915.6364
62,API3_exchange,API3,0.52,9971.8788
19,AUDIO_exchange,AUDIO,1.56,29915.6364
18,AVAX_exchange,AVAX,0.52,9971.8788
...,...,...,...,...
4,XRP_exchange,XRP,1.04,19943.7576
23,XTZ_exchange,XTZ,2.09,40079.2821
36,YGG_exchange,YGG,1.04,19943.7576
22,ZEC_exchange,ZEC,0.52,9971.8788


In [32]:
# second pivot table summarizes Coin and USD Value for the Exchange.

pivot_table_exchange = pd.pivot_table(exchange, values='USD Value', index='Coin', aggfunc='sum')

pivot_table_exchange.head(20)

,USD Value
Coin,
ADA,9971.8788
ALGO,29915.6364
API3,9971.8788
AUDIO,29915.6364
AVAX,9971.8788
BAKE,9971.8788
BLZ,9971.8788
BOND,9971.8788
BSW,9971.8788


In [248]:
#Pseudocode =

# Following this we need to compare which assets are exceeding individual product limits. 
# and how much many need their exchange position limit need increase.
# Compare between both USD Values of "pivot_table" dataframe and "pivot_table_exchange" dataframe.
